In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns=999

In [ ]:
train=pd.read_csv('../input/home-data-for-ml-course/train.csv')
train

x가 커지면 커질수록 y값도 커지는지 체크, 숫자적 칼럼있는지

In [ ]:
train.corr()['SalePrice'].sort_values()

In [ ]:
train.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
#모든 칼럼에 레이블링 한 번에 하는 법
#문자형 칼럼에 접근하는 방법
#결측치는 숫자형으로 받아들여진다. 
train.columns #모든 칼럼 출력할 수 있다. 
c=train.columns[train.dtypes==object] #조건 부여 옵션 
for i in c:
    train[i]=le.fit_transform(list(train[i])) #먼저 리스트처리 해주기, list만나면 형식이 카테고리화 된다. 우리 모델이 학습을 할 수 있음. 결측치도 다 의미있음! 
    #eg.보험데이터의 경우, 공란으로 둔 설문이 있으면, 데이터 수집 시 결측치로 나오겠지? 근데 이것도 정보로 체크해서 받아들임, 이것도 학습해야해,그래서 카테고리화 시켜준다. 
    print(i)

In [ ]:
train

In [ ]:
test=pd.read_csv('../input/home-data-for-ml-course/test.csv')
test

In [ ]:
c=test.columns[test.dtypes==object] #조건 부여 옵션 
for i in c:
    test[i]=le.fit_transform(list(test[i])) #먼저 리스트처리 해주기, list만나면 형식이 카테고리화 된다. 우리 모델이 학습을 할 수 있음. 결측치도 다 의미있음! 
    #eg.보험데이터의 경우, 공란으로 둔 설문이 있으면, 데이터 수집 시 결측치로 나오겠지? 근데 이것도 정보로 체크해서 받아들임, 이것도 학습해야해,그래서 카테고리화 시켜준다. 
    print(i)

In [ ]:
test

In [ ]:
k=pd.read_csv('../input/home-data-for-ml-course/sample_submission.csv')
k

In [ ]:
train_2=train.drop(columns=['Id','SalePrice'])
test_2=test.drop(columns=['Id'])

In [ ]:
train_2=train_2.fillna(0)
test_2=test_2.fillna(0)

In [ ]:
print(test_2.isnull().sum())

1.칼럼이 카테고리형이라 ->  tree 모델이 점수가 더 잘 나왔었음 
2. 전 처리시 숫자형으로 바꿨을 때 
3. 데이터 패턴을 더욱 체크해보기 

In [ ]:
#from sklearn.ensemble import RandomForestRegressor
#rf = RandomForestRegressor(n_jobs=4)
#rf.fit(train_2, train['SalePrice']) 
#result = rf.predict(test_2)
#result

xgboost -> 나무 생성할 때 마다 학습이 자꾸 발전됨 -> 벗 과대적합의 위험성이 있음 
옵션 하이퍼 파라미터 잘 설정해라 
1. learning rate 잘 설정해야함 (기본 값이 이상해서 0.3/ 너무 크다)
학습률: 모델이 얼마나 빠르게 학습할까를 정하는 것이다. 0.3은 놓지는 부분이 많음/ 세세하게 학습 ㄴㄴ 하게 됨
2. 결국 모델은 W 값을 잘 찾는게 중요하다. 어떤 input에도 / 지나쳐서 global optimal 을 놓치게 되니까유 -> 시간은 더 걸려도 최적의 값 찾을 수 있으니까
3. 0.1로만 하는 이유 너무 줄이면 학습 시 너무 모멘텀이 작아서 멈춰버림, local optimal에 안주해버림 
4. 칼럼들의 속성들이 거리기반인지(선형모델) / 카테고리식인지/supportvector machine/ knn  등 모델을 다양하게 사용해야한다. 
eg. 트리 모델은 구분 위주, 의사결정위주니까 숫자적인지 아닌지 체크

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(learning_rate=0.1) #이런 옵션들에 대해서 3~4개로 체크 
xgb.fit(train_2,train['SalePrice'])
result=xgb.predict(test_2)

In [ ]:
sub=pd.read_csv('../input/home-data-for-ml-course/sample_submission.csv')
sub['SalePrice']=result
sub

In [ ]:
sub.to_csv('house7.csv',index=0)